In [1]:
#part 1:  To get all Dec data _ keep original codes
import requests
import re
import pandas as pd
import pprint                  # Prints the nicely formatted dictionary
import datetime
#from datetime import datetime   # to do time calculation  #convert to datetime_object = datetime.strptime(datetime_str, '%m/%d/%y %H:%M:%S')
from datetime import timedelta

date_base = datetime.date(2020, 12, 1)
num_days = 31
df_list_all_date = []

for x in range(num_days):
    Date = date_base + datetime.timedelta(days=x)  
    print(str(Date), ' with type of :' , type(Date))
    url = "https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=" + str(Date) + "&lang=en&cargo=false&arrival=false"
    print(url)
    querystring = {"Date":str(Date)}     # Date: String: “YYYY-MM-DD” ; Arrival: (true=arrival, false=departure); Cargo (true=cargo, false=passenger)

    response = requests.request("GET", url, params=querystring)
    #pprint.pprint(response.text)
    df = pd.read_json(response.text)
    df_backup = df
    df1 = pd.DataFrame( df['list'][0])                  ### To turn the data into DataFrame & remove cancelled flight
    df1['dept_date']=Date
    df_temp = df1[df1['statusCode']=='DA']                #Remove::  CX = Cancelled , DU = Delayed to another day
    if x == 0:
        df_all_date = df_temp
    else:
        df_all_date= pd.concat([df_all_date , df_temp ])
    df_list_all_date.append(df_temp)
    ###print(df_temp)

    #df_temp.to_csv('airRaw' + str(Date) + '.csv' , index = None)  #---->to csv file 
#for y in range(1, df_all_date):
#df_all_date = pd.concat(df_list_all_date)
print(df_all_date)
df_all_date.to_csv('air' + '_Raw_202012v4' + '.csv' , index = None)

2020-12-01  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-01&lang=en&cargo=false&arrival=false
2020-12-02  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-02&lang=en&cargo=false&arrival=false
2020-12-03  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-03&lang=en&cargo=false&arrival=false
2020-12-04  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-04&lang=en&cargo=false&arrival=false
2020-12-05  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-05&lang=en&cargo=false&arrival=false
2020-12-06  with type of : <class 'datetime.date'>
https://www.hongkongairport.com/flightinfo-rest/rest/flights/past?date=2020-12-06&lang=en&cargo=false&arrival=fals

In [2]:
#part 2 - version 2:    To work on data based on Dec csv file as dataframe called df_temp
#doc:   https://www.hongkongairport.com/iwov-resources/misc/opendata/Flight_Information_DataSpec_en.pdf
#site:  https://data.gov.hk/en-data/dataset/aahk-team1-flight-info
import requests
import re
import pandas as pd
import numpy as np
import pprint                   
from datetime import datetime      
from datetime import timedelta  #add one day 
import json 

df_temp = df_all_date       #continue to work on the data above
#df_temp = pd.read_csv('air' + '_Raw_202012v4' + '.csv')

#---------------------------------------Data processing---------------------------------------
# destination column
df_temp['destination'] = df_temp['destination'].astype('str')  
df_temp['destination'].str.extract(r'([A-Z]+)',expand = True)

# flight number column
df_temp['flight number'] = df_temp['flight'].apply( lambda x: x[0]['no'])

# airline column
df_temp['airline'] = df_temp['flight'].apply( lambda x: x[0]['no'][0:2])        

#actual_dept_time column
df_temp['est_dept_time'] = df_temp['time'].apply( lambda x: datetime.strptime(x, '%H:%M'))
df_temp['day'] = df_temp['status'].apply( lambda x: timedelta(days=1) if '(' in x else timedelta(days=0) )
df_temp['actual_dept_time'] = df_temp['status'].str.extract(r'Dep (..:..)')
df_temp['actual_dept_time'] = df_temp['actual_dept_time'].apply( lambda x: datetime.strptime(x, '%H:%M'))  #convert to datetime format

#delay column
df_temp['delay_time'] = df_temp['actual_dept_time'] - df_temp['est_dept_time']  + df_temp['day']            
df_temp['delay_time'] = df_temp['delay_time'].apply( lambda x: x if x > timedelta(days=0) else timedelta(days=0))
#---------------------------------------Data processing (Done) -------------------------------

#---------------------------------------File export-------------------------------------------
df_temp.to_csv('air' + '_Finalizing_202012v4' + '.csv' , index = None)
df_temp.head()

,destination,terminal,aisle,gate,time,flight,statusCode,status,dept_date,flight number,airline,est_dept_time,day,actual_dept_time,delay_time
0,['FRA'],T1,B,64,00:35,"[{'no': 'CX 289', 'airline': 'CPA'}, {'no': 'L...",DA,Dep 01:09,2020-12-01,CX 289,CX,1900-01-01 00:35:00,0 days,1900-01-01 01:09:00,0 days 00:34:00
1,['AMS'],T1,B,47,00:50,"[{'no': 'CX 271', 'airline': 'CPA'}]",DA,Dep 01:08,2020-12-01,CX 271,CX,1900-01-01 00:50:00,0 days,1900-01-01 01:08:00,0 days 00:18:00
2,['LHR'],T1,,5,01:10,"[{'no': 'BA 032F', 'airline': 'BAW'}]",DA,Dep 01:34,2020-12-01,BA 032F,BA,1900-01-01 01:10:00,0 days,1900-01-01 01:34:00,0 days 00:24:00
3,['LHR'],T1,,45,01:30,"[{'no': 'BA 028F', 'airline': 'BAW'}]",DA,Dep 01:22,2020-12-01,BA 028F,BA,1900-01-01 01:30:00,0 days,1900-01-01 01:22:00,0 days 00:00:00
4,['TLV'],T1,B,26,01:30,"[{'no': 'CX 675', 'airline': 'CPA'}]",DA,Dep 01:21,2020-12-01,CX 675,CX,1900-01-01 01:30:00,0 days,1900-01-01 01:21:00,0 days 00:00:00


In [20]:
import requests
import re
import pandas as pd
import pprint                  
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait

URL = "https://www.worldairlineawards.com/worlds-top-100-airlines-2019/"
html = requests.get(URL)
soup = BeautifulSoup(html.text, "html.parser")
name_list = []
rating_list = []

for i in soup.find_all(class_="mb-0 text-responsive-h4"):
    name_list.append(i.text)
for i in soup.find_all(class_="mb-0 text-responsive-h4 font-weight-regular"):
    rating_list.append(i.text)
air_rate =dict(zip(name_list,rating_list))
df = pd.DataFrame(list(air_rate.items()),columns = ['Name','Rating'])
df

,Name,Rating
0,Qatar Airways,1
1,Singapore Airlines,2
2,ANA All Nippon Airways,3
3,Cathay Pacific Airways,4
4,Emirates,5
...,...,...
95,S7 Airlines,96
96,Royal Jordanian Airlines,97
97,Flynas,98
98,Air China,99


In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
sns.set(color_codes=True)

df = sns.load_dataset('df_temp')
df.head()

ValueError: 'df_temp' is not one of the example datasets.

In [42]:
import requests
URL = "https://www.iata.org/en/publications/directories/code-search/"
html = requests.get(URL)
soup = BeautifulSoup(html.text, "html.parser")
table = soup.find("table", attrs={"class":"datatable"})
#print(table)
# The first tr contains the field names.
headings = [td.get_text() for td in table.find("tr").find_all("td")]
datasets = []

import time, logging, threading

for x in range(1,80,1):
    link='?airline.page=' + str(x) + '&airline.search='
    URL = "https://www.iata.org/en/publications/directories/code-search/" + link
    #print(URL)
    html = requests.get(URL) #,timeout=(5,5)) #, timeout=5
    #time.sleep(5)
    #threading.Event.wait(5)
    #print(html)
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find("table", attrs={"class":"datatable"})
    for row in table.find_all("tr")[1:]:
        dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
        #print("dataset is: " ,list(dataset))
        datasets.append(list(dataset))
import pprint
#pprint.pprint(datasets)


full_name=[]
short_name=[]
for i in datasets:
    full_name.append(i[0][1])
    short_name.append(i[2][1])
destination_zipped1=dict(zip(short_name,full_name))
destination_zipped1

for x in range(80,160,1):
    link='?airline.page=' + str(x) + '&airline.search='
    URL = "https://www.iata.org/en/publications/directories/code-search/" + link
    #print(URL)
    html = requests.get(URL) #,timeout=(5,5)) #, timeout=5
    #time.sleep(5)
    #threading.Event.wait(5)
    #print(html)
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find("table", attrs={"class":"datatable"})
    for row in table.find_all("tr")[1:]:
        dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
        #print("dataset is: " ,list(dataset))
        datasets.append(list(dataset))

full_name=[]
short_name=[]
for i in datasets:
    full_name.append(i[0][1])
    short_name.append(i[2][1])
destination_zipped2=dict(zip(short_name,full_name))
destination_zipped1.update(destination_zipped2)
destination_zipped1

for x in range(160,219):
    link='?airline.page=' + str(x) + '&airline.search='
    URL = "https://www.iata.org/en/publications/directories/code-search/" + link
    #print(URL)
    html = requests.get(URL) #,timeout=(5,5)) #, timeout=5
    #time.sleep(5)
    #threading.Event.wait(5)
    #print(html)
    soup = BeautifulSoup(html.text, "html.parser")
    table = soup.find("table", attrs={"class":"datatable"})
    for row in table.find_all("tr")[1:]:
        dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
        #print("dataset is: " ,list(dataset))
        datasets.append(list(dataset))

full_name=[]
short_name=[]
for i in datasets:
    full_name.append(i[0][1])
    short_name.append(i[2][1])
destination_zipped3=dict(zip(short_name,full_name))
destination_zipped1.update(destination_zipped3)
destination_zipped1

 Cargo, Inc',
 'GF': 'Gulf Air B.S.C. (c)',
 'G2': 'Gullivair Ltd. dba Gullivair',
 'H6*': 'Hageland Aviation Services, Inc.',
 'HR': 'Hahn Air Lines GmbH',
 'H1': 'Hahn Air Systems',
 'X1': 'Hahn Air Technologies GmbH',
 'HU': 'Hainan Airlines Holding Company Limited',
 'YB': 'Harbour Air Ltd.',
 'HA': 'Hawaiian Airlines, Inc.',
 'NS': 'Hebei Airlines Co., Ltd.',
 'YO': 'Heli Air Monaco',
 'HS': 'Heli Securite Helicopter Airline',
 '2L': 'Helvetic Airways AG',
 'RX': 'HG Aviation Limited dba Regent Airways',
 '5K': 'Hi Fly Transportes Aereos, S.A.',
 'HG*': 'Hibernian Airlines',
 'H9': 'Himalaya Airlines Pvt. Ltd.',
 'OI': 'Hinterland Aviation Pty Ltd',
 'H4': 'HiSky Europe SRL',
 '1T': 'Hitit Bilgisayar Hizmetleri A.S.',
 '5Q': 'Holiday Europe Ltd.',
 'RH': 'Hong Kong Air Cargo Carrier Limited',
 'HX': 'Hong Kong Airlines Limited',
 'UO': 'Hong Kong Express Airways Limited',
 'A5': 'HOP!',
 'O2': 'Hopscotch Air, Inc',
 'QX': 'Horizon Air Industries, Inc.',
 'MR': 'Hunnu Air LLC',
 '9